<a href="https://colab.research.google.com/github/Ankan1998/Fire-Detection-with-Tensorflow-2.0/blob/master/Object_detection_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir Tensorflow

In [ ]:
!git -C /content/Tensorflow  clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 45937 (delta 8), reused 19 (delta 1), pack-reused 45911
Receiving objects: 100% (45937/45937), 551.02 MiB | 21.12 MiB/s, done.
Resolving deltas: 100% (31488/31488), done.


In [ ]:
%cd /content/Tensorflow/models/research
!protoc object_detection/protos/*.proto --python_out=.

/content/Tensorflow/models/research


In [ ]:
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

Processing /content/Tensorflow/models/research
     |████████████████████████████████| 8.6MB 12.4MB/s 
     |████████████████████████████████| 358kB 39.4MB/s 
     |████████████████████████████████| 849kB 38.6MB/s 
     |████████████████████████████████| 829kB 34.1MB/s 
     |████████████████████████████████| 153kB 43.7MB/s 
     |████████████████████████████████| 51kB 6.1MB/s 
     |████████████████████████████████| 71kB 8.9MB/s 
     |████████████████████████████████| 61kB 7.4MB/s 
     |████████████████████████████████| 63.8MB 128kB/s 
     |████████████████████████████████| 1.4MB 37.3MB/s 
     |████████████████████████████████| 81kB 8.6MB/s 
     |████████████████████████████████| 174kB 36.4MB/s 
     |████████████████████████████████| 102kB 11.2MB/s 
     |████████████████████████████████| 1.1MB 38.9MB/s 
     |████████████████████████████████| 36.7MB 53.4MB/s 
     |████████████████████████████████| 112kB 44.0MB/s 
  Created wheel for object-detection: filename=object_detection-

In [ ]:
import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

PATH_TO_SAVED_MODEL = "/content/drive/My Drive/Deep Learning/Object Detection/my_model/saved_model"

print('Loading model...', end='')
start_time = time.time()

# Load saved model and build the detection function
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

Loading model...Done! Took 24.795262336730957 seconds


In [ ]:
import numpy as np
from PIL import Image
from google.colab.patches import cv2_imshow

def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.
    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.
    Args:
      path: the file path to the image
    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path))

image_path = "/content/cullan-smith-BdTtvBRhOng-unsplash.jpg"
print('Running inference for {}... '.format(image_path), end='')

image_np = load_image_into_numpy_array(image_path)

# Things to try:
# Flip horizontally
# image_np = np.fliplr(image_np).copy()

# Convert image to grayscale, (You could uncomment this to try and see how the model reacts to a grayscale image)
# image_np = np.tile(
#     np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

# The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
input_tensor = tf.convert_to_tensor(image_np)
# The model expects a batch of images, so add an axis with `tf.newaxis`.
input_tensor = input_tensor[tf.newaxis, ...]

detections = detect_fn(input_tensor)

# All outputs are batches tensors.
# Convert to numpy arrays, and take index [0] to remove the batch dimension.
# We're only interested in the first num_detections.
num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
              for key, value in detections.items()}
detections['num_detections'] = num_detections

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_detections,
      detections['detection_boxes'],
      detections['detection_classes'],
      detections['detection_scores'],
      category_index=1,
      use_normalized_coordinates=True,
      max_boxes_to_draw=200,
      min_score_thresh=.1, # Adjust this value to set the minimum probability boxes to be classified as True
      agnostic_mode=False)

cv2_imshow(image_np_with_detections)

Running inference for /content/cullan-smith-BdTtvBRhOng-unsplash.jpg... 